# Object-Oriented Inheritance

A key concept of OOP in Python and other languages is the notion of inheritance, also known as subclassing or subtyping. Consider modeling Animals:

In [1]:
class Animal:

    def __init__(self, name):
        self._name = name
        
    def speak(self, message):
        print(f'{self._name} the {type(self).__name__} says "{message}"')
        
    def get_number_of_legs(self):
        raise NotImplementedError

In [2]:
animal = Animal('Generic')

In [3]:
animal.speak('Can I really speak?')

Generic the Animal says "Can I really speak?"


In [4]:
animal.get_number_of_legs()

NotImplementedError: 

We might want to create some more specific `class`es that "know" how many legs they have. We do this by **overriding** the implementation of the `get_number_of_legs` method:

In [5]:
class Biped(Animal):   # "Biped extends Animal" or "Biped specializes Animal"
    
    def get_number_of_legs(self):
        return 2

    
class Quadruped(Animal):
    
    def get_number_of_legs(self):
        return 4  

override ~== overwrite

In [6]:
monkey = Biped('George')
dog = Quadruped('Gracie')

In [7]:
isinstance(monkey, Biped)

True

In [8]:
isinstance(monkey, Animal)

True

is-a vs has-a

In [9]:
issubclass(Biped, Animal)

True

In [10]:
isinstance(Biped, Animal)

False

In [11]:
isinstance(Biped, type)

True

In [12]:
monkey.speak('hello')

George the Biped says "hello"


In [13]:
dog.speak('hello')

Gracie the Quadruped says "hello"


In [14]:
monkey.get_number_of_legs()

2

In [15]:
dog.get_number_of_legs()

4

We can further specialize our classes. In this case, we'll provide a default name for instances of the `Monkey` and `Dog` class, and then **delegate** to the base class ("superclass") implementation of `__init__`:

Keep your code "DRY" -- 

- Don't 
- Repeat
- Yourself

In [16]:
class Monkey(Biped):

    def __init__(self, name='George'):
        super().__init__(name)
        # Biped.__init__(self, name) not preferred
        

class Dog(Quadruped):
    
    def __init__(self, name='Gracie'):
        super().__init__(name)

In [17]:
animals = [Monkey(), Dog(), Dog('Fido')]

In [18]:
for a in animals:
    a.speak(f'Hello there, I have {a.get_number_of_legs()} legs')
#     f = a.speak  => "bound method"
#     f('Hello there, I have {a.get_number_of_legs()} legs')    

George the Monkey says "Hello there, I have 2 legs"
Gracie the Dog says "Hello there, I have 4 legs"
Fido the Dog says "Hello there, I have 4 legs"


# Python attribute lookup

Assume we have this code:

```python
foo = Dog()
```

When you use the syntax `foo.bar`, or `getattr(foo, "bar")` what does Python actually _do_?

1. Examine the **instance** foo to see if has an instance attribute named `bar`. If it does, return it.
1. Examine the the **class** of foo (in this case, `Dog`) to see if it has an attribute named `bar`. If it does, return it.
1. Using the *method resolution order* (MRO) of Dog, examine the superclasses (a.k.a. base classes, ancestor classes, etc.) of Dog to see if any of them have an attribute named `bar`, returning the value if it does
1. Upon exhausting the MRO, without finding the name, raise `AttributeError`

This happens on all attribute lookups (i.e. reading a dotted name, or using the builtin function `getattr()`), including method lookups. 

In [19]:
def tripod():
    return 3

animals[2].get_number_of_legs = tripod

In [20]:
animals[2].get_number_of_legs()

3

In [21]:
animals[1].get_number_of_legs()

4

In [22]:
Dog.mro()

[__main__.Dog, __main__.Quadruped, __main__.Animal, object]

In [23]:
a.__dict__

{'_name': 'Fido', 'get_number_of_legs': <function __main__.tripod()>}

In [24]:
type(a)

__main__.Dog

In [25]:
type(a).__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Dog.__init__(self, name='Gracie')>,
              '__doc__': None})

In [26]:
Quadruped.__dict__

mappingproxy({'__module__': '__main__',
              'get_number_of_legs': <function __main__.Quadruped.get_number_of_legs(self)>,
              '__doc__': None})

In [27]:
a.speak

<bound method Animal.speak of <__main__.Dog object at 0x7ff05c3f9100>>

In [28]:
a.get_number_of_legs   # not really a method 

<function __main__.tripod()>

In [29]:
getattr(a, '_name')  # a._name

'Fido'

In [30]:
Monkey.mro()

[__main__.Monkey, __main__.Biped, __main__.Animal, object]

In [31]:
Dog.mro()

[__main__.Dog, __main__.Quadruped, __main__.Animal, object]

## Multiple Inheritance

The MRO if a class in a single-inheritance situation is just a linear search. If we use multiple inheritance, the situation is a bit more complex. 

In [32]:
class MonkeyDog(Monkey, Dog):
    pass

class DogMonkey(Dog, Monkey):
    pass

```
Animal
|  \
.    .
Dog  Monkey
|   /
| /
DogMonkey```

In [33]:
weird = [MonkeyDog(), DogMonkey()]

In [34]:
for a in weird:
    a.speak(f'Hello there, I have {a.get_number_of_legs()} legs')    

George the MonkeyDog says "Hello there, I have 2 legs"
Gracie the DogMonkey says "Hello there, I have 4 legs"


### MRO

Two (+1) desirable properties:

- visit each superclass exactly once
- visit a subclass before its superclass
- (+1) linearization

In [35]:
MonkeyDog.mro()

[__main__.MonkeyDog,
 __main__.Monkey,
 __main__.Biped,
 __main__.Dog,
 __main__.Quadruped,
 __main__.Animal,
 object]

In [36]:
DogMonkey.mro()

[__main__.DogMonkey,
 __main__.Dog,
 __main__.Quadruped,
 __main__.Monkey,
 __main__.Biped,
 __main__.Animal,
 object]

### Multiple inheritance and `super()`

`super()` also obeys the MRO of the class when finding the superclass, so things can sometimes get confusing:

In [37]:
class Animal():

    def __init__(self, name):
        self._name = name
        
    def speak(self, message):
        print(f'{self._name} the {type(self).__name__} says "{message}"')
        
    def get_number_of_legs(self):
        raise NotImplementedError
        
    def illustrate(self):
        print('In Animal.illustrate')
        
        
class Biped(Animal):
    
    def get_number_of_legs(self):
        return 2

    def illustrate(self):
        print('In Biped.illustrate, delegating to super')
        super().illustrate()
        
    
class Quadruped(Animal):
    
    def get_number_of_legs(self):
        return 4 

    def illustrate(self):
        print('In Quadruped.illustrate, delegating to super')
        super().illustrate()
    

class Monkey(Biped):
    
    def __init__(self, name='George'):
        super().__init__(name)

    def illustrate(self):
        print('In Monkey.illustrate, delegating to super')
        super().illustrate()
        

class Dog(Quadruped):
    
    def __init__(self, name='Fido'):
        super().__init__(name)

    def illustrate(self):
        print('In Dog.illustrate, delegating to super')
        super().illustrate()
        
        
class MonkeyDog(Monkey, Dog):
    
    def illustrate(self):
        print('In MonkeyDog.illustrate, delegating to super')
        super().illustrate()


class DogMonkey(Dog, Monkey):

    def illustrate(self):
        print('In DogMonkey.illustrate, delegating to super')
        super().illustrate()

In [38]:
md = MonkeyDog()
dm = DogMonkey()
m = Monkey()
d = Dog()

In [39]:
m.illustrate()

In Monkey.illustrate, delegating to super
In Biped.illustrate, delegating to super
In Animal.illustrate


In [40]:
d.illustrate()

In Dog.illustrate, delegating to super
In Quadruped.illustrate, delegating to super
In Animal.illustrate


In [41]:
md.illustrate()

In MonkeyDog.illustrate, delegating to super
In Monkey.illustrate, delegating to super
In Biped.illustrate, delegating to super
In Dog.illustrate, delegating to super
In Quadruped.illustrate, delegating to super
In Animal.illustrate


In [42]:
dm.illustrate()  # Python 2.7 & below, "super(Quadruped, self)"

In DogMonkey.illustrate, delegating to super
In Dog.illustrate, delegating to super
In Quadruped.illustrate, delegating to super
In Monkey.illustrate, delegating to super
In Biped.illustrate, delegating to super
In Animal.illustrate


In [43]:
def superish(method_class, self):
    mro = type(self).mro()
    for i, cls  in enumerate(mro):
        if cls == method_class:
            break
    return mro[i+1]

In [44]:
superish(Quadruped, dm)

__main__.Monkey

In [45]:
superish(Quadruped, d)

__main__.Animal

## Example: SocketServer

Multiple dimensions of abstraction

- Protocol: TCP vs UDP vs Unix
- Concurrency: Threads vs Processes

Solution: (at least in stdlib): (similar to this)...
```python
class SocketServer
class UDPServer(SocketServer)
class TCPServer(SocketServer)
class ThreadingMixin
class ForkingMixin
class ThreadingTCPServer(ThreadingMixin, TCPServer): pass
```

In [46]:
import socketserver
socketserver.ThreadingTCPServer??

In [47]:
socketserver.ThreadingMixIn??

In [48]:
class PrettyMixin:
    repr_format = 'detail goes here'

    def __repr__(self):
        detail = self.repr_format.format(self=self)
        return f'<{type(self).__qualname__} {detail}>'

In [49]:
pm = PrettyMixin()
pm

<PrettyMixin detail goes here>

In [50]:
class SQLTable():
    pass

class SomethingElse(PrettyMixin, SQLTable):
    repr_format = 'name={self.name}'
    
    def __init__(self):
        self.name = 'Rick'

In [51]:
se = SomethingElse()
se

<SomethingElse name=Rick>

# Lab

Open [OOP inheritance lab][oop-inheritance-lab]

[oop-inheritance-lab]: ./oop-inheritance-lab.ipynb